# k-means clustering

In this exercise we will apply the concepts learned during the lectures regarding unsupervised clustering methods.

The k-means algorithm is a simple clustering approach that can still be very effective under the right circumstances, i.e. when the assumptions of k-means are met.

First we will load the necessary packages and select a suitable colormap.

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib
from matplotlib import pyplot as plt

# set nipy_spectral as default colormap for all plots from now on
matplotlib.rc('image', cmap='nipy_spectral')

#### Task 1

Load and inspect the provided 2-dimensional data set `synth.txt` and determine a suitable value for $k$ before we apply the actual algorithm in task 2. If you are not sure, pick multiple feasible values for $k$ that could be used.

In [ ]:
# Your code below here
X = np.loadtxt('data/synth.txt')
plt.scatter(X[:, 0], X[:, 1], alpha=.03)

#### Solution

We can guess that 15 might be a suitable number of clusters. Note that this synthetic dataset was generated from 15 different distributions which confirms our guess. However, for real-world data sets, we generally do not know this ground truth.

#### Task 2

a) Apply the `KMeans` algorithm as provided by sklearn to the data set using your chosen values for $k$. Additionally use the values 2 and 100 for $k$.

b) For each value of $k$, visualize your results. Use a scatter plot with each cluster having a distinct color. Additionally plot the cluster centroids. As you will be plotting multiple cluster results in the next few tasks, writing a `plot_clustering` function may be helpful. *Hint:* You can access the cluster labels from the fitted kmeans object using `kmeans.labels_` and the centroids using `kmeans.cluster_centers_`.


In [ ]:
# Your code below here
def plot_clustering(data, labels, centroids=None, title='', ax=None):
    if data.shape[1] != 2:
        raise ValueError('Only two-dimensional data is supported.')
    if ax is None:
        _, ax = plt.subplots(1,1)
    ax.set_title(title)
    ax.scatter(data[:, 0], data[:, 1], c=labels)
    if centroids is not None:
        ax.scatter(centroids[:, 0], centroids[:, 1], c='magenta', s=150, marker='+')

In [ ]:
# Insert your values for k you would like to test k-means with into this array
good_Ks = [2, 15, 100]
# Your code below here
for K in good_Ks:
    np.random.seed(42)  # ensure reproducible results
    kmeans = KMeans(n_clusters=K).fit(X)
    plot_clustering(X, kmeans.labels_, centroids=kmeans.cluster_centers_, title=f'$K={K}$')

# Assessing clustering quality

As the used data set was only using $D=2$ dimensions, we could easily plot the results and judge whether the assigned clusters make sense. For large values of $D$, a visual inspection would require either a dimensionality reduction as preprocessing (e.g. via PCA), or a post-processing (e.g. via tSNE, see https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) to project the clustered data to a lower-dimensional representation. However, there are metrics that quantify how *good* a clustering results is. One commonly used metric is the **silhouette score**. This metric quantifies the intuition, that data points within one cluster should be close to each other but far away from neighbouring clusters.

For each single data point $i$ in a cluster $C_n$ we can calculate how well this data point fits within its cluster, i.e. whether the average distance of this $i$-th data point in cluster $C_n$ to all other data points in $C_n$ is small.

$a(i) = \dfrac{1}{|C_n| - 1} \sum\limits_{j \in C_n, i\neq j} d(i,j), \;\; i,j \in C_n$

Any distance measure $d$ can be used, however, we use Euclidean distance. This metric $a(i)$ quantifies how well a data point fits within its assigned cluster. Now we want to quantify how dissimilar this data point $i$ is to the other clusters. Specifically we are interested in the dissimilarity to its closest neighbouring cluster.

$b(i) = \min\limits_{C_m \neq C_n} \dfrac{1}{|C_m|} \sum\limits_{j \in C_m} d(i,j)$

Using both of these metrics we can now formulate the silhouette score:

$s(i) = \dfrac{b(i) - a(i)}{\max\{a(i), b(i)\}}$

Note that $s(i) \in \left[-1,1\right]$ where $-1$ represents a bad cluster assignment and $1$ a good assignment.


#### Task 3

Use the `silhouette_score` implementation of sklearn to calculate the average silhouette score of all samples for a clustering result.

Calculate the average silhouette score of all data points for clusterings obtained by using the k-means algorithm for all values of $k \in [2,20]$. Finally, plot the average silhouette score depending on $k$. How would you interpret these results?

In [ ]:
from sklearn.metrics import silhouette_score

Ks = range(2,21)
# Your code below here
scores = []
for K in Ks:
    np.random.seed(42)
    kmeans = KMeans(n_clusters=K).fit(X)
    score = silhouette_score(X, kmeans.labels_)
    print(f'Silhouette score for {K} clusters: {score:1.3f}')
    scores.append(score)

plt.plot(Ks, scores)
plt.xlabel('K clusters')
plt.ylabel('Average silhouette score')
plt.title('Silhouette score for $K$ clusters')

#### Solution

If we had to use a small value for $k$, a value of 4 would probably be most appropriate, as this yields a local maximum of silhouette scores. However, for the given range, $k=15$ yields the best silhouette score, which makes sense, as the data was generated from 15 different distributions.